<a href="https://colab.research.google.com/github/stevendemer/machine-learning/blob/main/assignment3_24_25(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Εργασία 3 ##

Καλωσήρθατε στην τρίτη σας εργασία. Η εργασία αυτή έχει σκοπό να σας βοηθήσει να εμπεδώσετε τα σύνολα μοντέλων.

Στην εργασία αυτή θα πρέπει να συμπληρώσετε κώδικα Python 3 στα σημεία που αναφέρουν # YOUR CODE HERE. Μην τροποποιείτε τον κώδικα που βρίσκεται εκτός αυτών των περιοχών.

Πρωτού παραδόσετε την εργασία σας σιγουρευτείτε ότι ο κώδικας σε όλα τα κελιά τρέχει σωστά. Για το σκοπό αυτό επιλέξτε από το μενού **Χρόνος εκτέλεσης (runtime) -> Επανεκίνηση περιόδου λειτουργίας και εκτέλεση όλων**.

Συμπληρώστε το όνομα (**NAME**) και το **AEM** σας παρακάτω **εντός των διπλών εισαγωγικών**:

In [ ]:
NAME = "Stefanos Demertzis"
AEM = "3467"

**1 (2 μονάδες)** Σε αυτήν την εργασία θα δουλέψουμε με ένα σύνολο δεδομένων δυαδικής ταξινόμησης, από το δημόσιο αποθετήριο δεδομένων και πειραμάτων μηχανικής μάθησης [openml.org](https://www.openml.org/). Μπορείτε να δείτε πως φορτώνουμε σύνολα δεδομένων από το openml.org μέσα από την sklearn μελετώντας την αντίστοιχη [τεκμηρίωση](https://scikit-learn.org/stable/datasets/loading_other_datasets.html#downloading-datasets-from-the-openml-org-repository). Συγκεκριμένα θέλουμε να δουλέψουμε με την έκδοση 1 του Pima Indians Diabetes Database, το οποίο αφορά τη διάγνωση διαβήτη. Χωρίστε το σύνολο δεδομένων σε δεδομένα εκπαίδευσης (X_train, y_train) και ελέγχου (X_test, y_test) σε ποσοστό 70% και 30% αντίστοιχα με τη συνάρτηση train_test_split (τιμή για random_state βάλτε 0).

In [40]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

X, y = fetch_openml('diabetes', version=1, as_frame=True, return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0
)
y_train = (y_train == 'tested_positive').astype(int)
y_test = (y_test == 'tested_positive').astype(int)


In [41]:
"""Τεστ ορθής ανάγνωσης και διαχωρισμού του συνόλου δεδομένων"""
assert X_train['mass'][0] == 33.6
assert X_test['pedi'][34] == 0.512

**2 (4 μονάδες)** Υλοποιήστε μια ντετερμινιστική εκδοχή της μεθόδου των τυχαίων υποχώρων, η οποία χτίζει τόσα μοντέλα όσες και οι μεταβλητές εισόδου, κάθε ένα από τα οποία αγνοεί και μία διαφορετική μεταβλητή εισόδου. Π.χ. το πρώτο μοντέλο αγνοεί την πρώτη, το δεύτερο αγνοεί τη δεύτερη κτλ. Χρησιμοποιήστε τη συνάρτηση clone από το sklearn.base για να δημιουργήστε αντίγραφο του βασικού μοντέλου σε κάθε επανάληψη.

In [42]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone
from scipy.stats import mode

class RandomSubspaceDet:
    def __init__(self, estimator=DecisionTreeClassifier()):
        self.estimator = estimator
        self.models = []
        self.feature_indices = []


    def fit(self, X_train, y_train):
        n_features = X_train.shape[1]
        self.models = []
        self.feature_indices = []

        for i in range(n_features):
          # use all the features except the i-th one
          cols = [j for j in range(n_features) if j != i]
          X_train_subset = X_train.iloc[:, cols]

          model = clone(self.estimator)
          model.fit(X_train_subset, y_train)
          self.models.append(model)
          self.feature_indices.append(cols)


    def predict(self, X):
        predictions = []
        for i in range(len(self.models)):
          X_subset = X.iloc[:, self.feature_indices[i]]
          prediction = self.models[i].predict(X_subset)
          # Ensure predictions are integers before appending
          prediction = (prediction == 'tested_positive').astype(int) if prediction.dtype.kind in {'U', 'O'} else prediction
          predictions.append(prediction)


        predictions = np.array(predictions)
        majority_votes = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=predictions)
        return majority_votes

In [43]:
"""Τεστ ορθής υλοποίησης RandomSubspaceDet"""
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

rs = RandomSubspaceDet(estimator=DecisionTreeClassifier(random_state=1))
rs.fit(X_train, y_train)
assert round(accuracy_score(rs.predict(X_test), y_test), 4) == 0.7446

**3 (4 μονάδες)** Υλοποιήστε τη μέθοδο AdaBoost όπως παρουσιάστηκε στο μάθημα. Χρησιμοποιήστε τη συνάρτηση clone από το sklearn.base για να δημιουργήστε αντίγραφο του βασικού μοντέλου σε κάθε επανάληψη. Χρησιμοποιήστε την παράμετρο sample_weight της fit του βασικού μοντέλου για να ορίσετε τα βάρη των παραδειγμάτων εκπαίδευσης.

In [44]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone

class AdaBoost:
    def __init__(self, n_estimators=20, estimator=DecisionTreeClassifier(max_depth=1)):
        self.estimator = estimator
        self.n_estimators = n_estimators
        self.models = []
        self.alphas = []


    def fit(self, X_train, y_train):
        n_samples = X_train.shape[0]

        w = np.full(n_samples, 1 / n_samples)

        self.models = []
        self.alphas = []

        for _ in range(self.n_estimators):

            model = clone(self.estimator)

            model.fit(X_train, y_train, sample_weight=w)

            y_pred = model.predict(X_train)


            miss = (y_pred != y_train)
            err = np.sum(w * miss) / np.sum(w)
            if err == 0:

                alpha = 1
                self.models.append(model)
                self.alphas.append(alpha)
                break
            elif err >= 0.5:

                continue


            alpha = 0.5 * np.log((1 - err) / err)


            w = w * np.exp(alpha * miss) * np.exp(-alpha * (1 - miss))


            w /= np.sum(w)

            self.models.append(model)
            self.alphas.append(alpha)




    def predict(self, X):

        model_preds = np.array([model.predict(X) for model in self.models])

        model_preds = np.where(model_preds == 0, -1, 1)


        weighted_sum = np.dot(self.alphas, model_preds)


        return np.where(weighted_sum >= 0, 1, 0)

In [45]:
"""Τεστ ορθής υλοποίησης AdaBoost"""
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

ab = AdaBoost(n_estimators=20, estimator=DecisionTreeClassifier(max_depth=2, random_state=1))
ab.fit(X_train, y_train)
assert round(accuracy_score(ab.predict(X_test), y_test), 4) == 0.7706


In [46]:
#Ίδιο αποτέλεσμα παίρνουμε και με την κλάση της sklearn. Ωστόσο υπάρχουν γενικότερα διαφορές στην υλοποίηση της σε σ΄χέση με αυτήν που είδαμε στη θεωρία, όπως η υποστήριξη παραπάνω από 2 κλάσεων και ο διαφορετικός χειρισμός της τυχαιότητας στην εκπαίδευση των επί μέρους μοντέλων.
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

ab = AdaBoostClassifier(n_estimators=20, estimator=DecisionTreeClassifier(max_depth=2, random_state=1))
ab.fit(X_train, y_train)
assert round(accuracy_score(ab.predict(X_test), y_test), 4) == 0.7706